In [1]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

In [2]:
import minsearch
import json
import anthropic

from IPython.display import Markdown, display
from anthropic import HUMAN_PROMPT, AI_PROMPT

In [3]:
client = anthropic.Anthropic(api_key="your-api-key")

In [4]:
# !wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [7]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [8]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [9]:
q = 'the course has already started, can I still enroll?'

In [10]:
index.fit(documents)

In [14]:
response = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": q}
    ]
)
Markdown(response.content[0].text)

I apologize, but I don't have any information about the specific course you're referring to or its enrollment policies. The best way to find out if you can still enroll in the course is to directly contact the institution or department offering the course. They will be able to provide you with accurate and up-to-date information about their enrollment deadlines and any possible exceptions.

In general, many courses allow for late enrollment within a certain timeframe after the start date, but this varies greatly depending on the institution and the type of course. Some factors that may affect your ability to enroll after the course has begun include:

1. The course's late enrollment policies
2. The amount of coursework you would need to catch up on
3. The availability of space in the course
4. Any prerequisites or requirements you need to fulfill

Remember, it's always best to reach out to the course provider directly to get the most accurate and relevant information for your specific situation.

In [32]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )
    return results

In [44]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QEUSTION based on CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE.
    
    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()

    return prompt

In [39]:
def llm(prompt):
    response = client.messages.create(
        model="claude-3-opus-20240229",
        max_tokens=1024,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return Markdown(response.content[0].text)

In [50]:
query1 = 'the course has already started, can I still enroll?'
query2 = 'how do I run kafka?'

In [49]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [51]:
rag(query1)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

In [52]:
rag(query2)

To run Kafka, first ensure that the necessary Docker images are up and running. Then, depending on the programming language you are using:

For Java:
In the project directory, run:
java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java

For Python: 
Create a virtual environment, activate it, and install the required packages from requirements.txt:
python -m venv env
source env/bin/activate
pip install -r ../requirements.txt

Then you can run the Python Kafka producer/consumer files within this virtual environment.

Note: Due to issues with the kafka-python package, it is recommended to use kafka-python-ng instead:
pip install kafka-python-ng